In [31]:
import duckdb
from datetime import datetime, timedelta
con = duckdb.connect("data/db.duckdb", read_only=False)

In [32]:
# dates
today = datetime(2024, 4, 30)
seven_days_ago = (today - timedelta(days=7)).strftime("%Y-%m-%d")
thirty_days_ago = (today - timedelta(days=30)).strftime("%Y-%m-%d")

# if table exists, then drop
con.sql("DROP TABLE IF EXISTS crm.customer_buying_intent")

# creating table + query
con.sql("""
CREATE TABLE crm.customer_buying_intent AS (
    SELECT c.id as company_id,
           CASE
             WHEN EXISTS(
                    SELECT 1
                    FROM   crm.website_visits wv
                    WHERE  Substring(email FROM Position('@' IN email) + 1) = c.domain
                           AND wv.event_date_utc >= ?
                 )
                  AND EXISTS (
                    SELECT 1
                    FROM   crm.news_publications np
                    WHERE  REPLACE(REPLACE(REPLACE(company_website, 'http://', ''), 'https://', ''), 'www.', '') = c.domain
                           AND np.publication_date_utc >= ?
                 ) THEN 'HIGH'
             WHEN EXISTS(
                    SELECT 1
                    FROM   crm.news_publications np
                    WHERE  REPLACE(REPLACE(REPLACE(company_website, 'http://', ''), 'https://', ''), 'www.', '') = c.domain
                           AND np.publication_date_utc >= ?
                 ) THEN 'MEDIUM'
             ELSE 'LOW'
           END AS buying_intent
    FROM   crm.companies c
)
""", params=[seven_days_ago, seven_days_ago, thirty_days_ago])


In [33]:
con.commit() 
con.sql(
"""
SELECT * FROM crm.customer_buying_intent
""")

┌──────────────────────────────────────┬───────────────┐
│              company_id              │ buying_intent │
│               varchar                │    varchar    │
├──────────────────────────────────────┼───────────────┤
│ 5355564e-bf45-4e91-b17a-3394349bf1fa │ MEDIUM        │
│ 5a121ab4-94d3-4edb-8e85-df27753a260f │ LOW           │
│ 2faf7069-e82d-4763-a85a-11eea6c3ae87 │ LOW           │
│ 9267da0b-2a50-4859-9203-3cb6c375bedc │ MEDIUM        │
│ 57fb99b9-921e-45f6-af5e-01e297b46d50 │ LOW           │
│ db0076e7-2cbd-4487-b901-36dfb30022e0 │ MEDIUM        │
│ 6b0e6cc9-c0b1-4cc6-9651-511ccae9d2c3 │ LOW           │
│ 8da397da-e1fa-4f03-ac32-4ec0d1440f8a │ MEDIUM        │
│ d79c31c7-e8f5-4a99-85df-6489d2cc93de │ LOW           │
│ 3ca29658-233f-4f74-bbd1-a23ea9960709 │ LOW           │
│                  ·                   │  ·            │
│                  ·                   │  ·            │
│                  ·                   │  ·            │
│ 2d37dde9-2238-4b46-92cf-5b39e

In [34]:
con.close()